In [1]:
%run Main.ipynb

# Variation of the fine-structure constant

## - Models
**Bekenstein’s class of varying $\alpha$ models $\to$ flat $\Lambda$CDM case**

$\frac{\Delta \alpha}{\alpha}(z) = 2 \zeta_m \log(1+z) + \frac{2(\zeta_{\Lambda} - 2\zeta_m)}{3\sqrt{\Omega_{\Lambda}}} \left[ \log\left( \frac{1+\sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_m}}\right) - \sqrt{E^2(z)} \log\left( \frac{\sqrt{\Omega_{\Lambda}} + \sqrt{E^2(z)}}{\sqrt{\Omega_m (1+z)^3}} \right) \right] \Leftrightarrow$ 

$\Leftrightarrow \frac{\Delta \alpha}{\alpha}(z) = \left[ \frac{2}{3\sqrt{\Omega_{\Lambda}}} \left[ \log\left( \frac{1+\sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_m}}\right) - \sqrt{E^2(z)} \log\left( \frac{\sqrt{\Omega_{\Lambda}} + \sqrt{E^2(z)}}{\sqrt{\Omega_m (1+z)^3}} \right) \right] \right] \zeta_{\Lambda} + \left[ 2\log(1+z) - \frac{4}{3\sqrt{\Omega_{\Lambda}}} \left[ \log\left( \frac{1+\sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_m}}\right) - \sqrt{E^2(z)} \log\left( \frac{\sqrt{\Omega_{\Lambda}} + \sqrt{E^2(z)}}{\sqrt{\Omega_m (1+z)^3}} \right) \right] \right] \zeta_m$

For a flat universe we have: $\Omega_m + \Omega_{\Lambda} = 1 \Rightarrow \Omega_{\Lambda} = 1 - \Omega_m$

In [2]:
def zeta_lambda(z, OmegaM, w0, wa):
    if w0 == None: 
        w0 = -1.
    if wa == None: 
        wa = 0
    #in flat ΛCDM case we always have w0=-1 and wa=0
    zeta_lambda = 2/(3*np.sqrt(1-OmegaM)) * (np.log((1+np.sqrt(1-OmegaM))/np.sqrt(OmegaM)) - 
     (function_E_z(z, OmegaM, w0, wa)*np.log((np.sqrt(1-OmegaM) + 
        function_E_z(z, OmegaM, w0, wa))/(np.sqrt(OmegaM * (1+z)**3)))))
    return zeta_lambda

def zeta_m(z, OmegaM, w0, wa):
    if w0 == None: 
        w0 = -1.
    if wa == None: 
        wa = 0
    #in flat ΛCDM case we always have w0=-1 and wa=0
    zeta_m = 2*np.log(1+z) - (4/(3*np.sqrt(1-OmegaM))) * (np.log((1+np.sqrt(1-OmegaM))/np.sqrt(OmegaM)) - 
     (function_E_z(z, OmegaM, w0, wa)*np.log((np.sqrt(1-OmegaM) + 
        function_E_z(z, OmegaM, w0, wa))/(np.sqrt(OmegaM * (1+z)**3)))))
    return zeta_m

def alpha_v_bekenstein(z, OmegaM, w0, wa, Zl, Zm):
    return zeta_lambda(z, OmegaM, w0, wa) * Zl + zeta_m(z, OmegaM, w0, wa) * Zm

**CPL Parametrization**

$w_{CPL}(z) = w_0 + w_a \frac{z}{1+z} $ and $\Omega_{CPL}(z) = \frac{1-\Omega_m}{1-\Omega_m+\Omega_m (1+z)^{-3(w_0+w_a)} \exp(3w_az/(1+z))}$

For $w_\phi\geq-1$:

$\frac{\Delta \alpha}{\alpha}(z) = \zeta \int_{0}^{z} \sqrt{3\Omega_\phi(z') [1+w_\phi(z')]} \,\frac{dz\prime}{1+z\prime}$

For Phantom Energy Fields $(w_\phi<-1)$:

$\frac{\Delta \alpha}{\alpha}(z) = -\zeta \int_{0}^{z} \sqrt{3\Omega_\phi(z') \mid 1+w_\phi(z')\mid} \,\frac{dz'}{1+z'}$

In [3]:
def w_CPL(z, w0, wa):
    w_CPL = w0 + wa * (z/(1+z))
    return w_CPL

def Omega_CPL(z, OmegaM, w0, wa):
    Omega_CPL = (1 - OmegaM) / (1 - OmegaM + (OmegaM*(1+z)**(-3*(w0+wa)) * np.exp((3*wa*z)/(1+z))))
    return Omega_CPL


def function(z, OmegaM, w0, wa):
    if w0 >= -1:            
        return np.sqrt(3*Omega_CPL(z, OmegaM, w0, wa) * (1 + w_CPL(z, w0, wa)))/(1+z)
    if 3*Omega_CPL(z, OmegaM, w0, wa) * (1 + w_CPL(z, w0, wa)) < 0:
        return -np.sqrt(3*Omega_CPL(z, OmegaM, w0, wa) * abs(1 + w_CPL(z, w0, wa)))/(1+z)


def alpha_v_CPL(z, OmegaM, w0, wa, zeta):
    alpha_v_CPL, error = scipy.integrate.quad(function, 0, z, args=(OmegaM, w0, wa),epsabs=10**(-30))
    return zeta * alpha_v_CPL  

## - Derivatives calculation
**Bekenstein**

In [4]:
def dfdOmegaM_ab(z, OmegaM, w0, wa, Zl, Zm):
    return ( alpha_v_bekenstein(z, OmegaM+hh, w0, wa, Zl, Zm)-alpha_v_bekenstein(z, OmegaM, w0, wa, Zl, Zm))/hh

def dfdZl(z, OmegaM, w0, wa, Zl, Zm):
    return ( alpha_v_bekenstein(z, OmegaM, w0, wa, Zl+hh, Zm)-alpha_v_bekenstein(z, OmegaM, w0, wa, Zl, Zm))/hh

def dfdZm(z, OmegaM, w0, wa, Zl, Zm):
    return ( alpha_v_bekenstein(z, OmegaM, w0, wa, Zl, Zm+hh)-alpha_v_bekenstein(z, OmegaM, w0, wa, Zl, Zm))/hh

**CPL Parametrization**

In [5]:
def dfdOmegaM_a(z, OmegaM, w0, wa, zeta):
    return ( alpha_v_CPL(z, OmegaM+hh, w0, wa, zeta)-alpha_v_CPL(z, OmegaM, w0, wa, zeta) )/hh

def dfdw0_a(z, OmegaM, w0, wa, zeta):
    return ( alpha_v_CPL(z, OmegaM, w0+hh, wa, zeta)-alpha_v_CPL(z, OmegaM, w0, wa, zeta) )/hh

def dfdwa_a(z, OmegaM, w0, wa, zeta):
    return ( alpha_v_CPL(z, OmegaM, w0, wa+hh, zeta)-alpha_v_CPL(z, OmegaM, w0, wa, zeta) )/hh

def dfdzeta(z, OmegaM, w0, wa, zeta):
    return ( alpha_v_CPL(z, OmegaM, w0, wa, zeta+hh)-alpha_v_CPL(z, OmegaM, w0, wa, zeta) )/hh

## - Add Priors

#### Atomic Clocks

- Considering $\Omega_m$ as a constant:

Present-day drift rate constrained by laboratory tests using atomic clocks $(\Omega_m = 0.3)$:

$\left( \frac{1}{H}\frac{\dot{\alpha}}{\alpha} \right)_0 = - 0.13 \zeta_{\Lambda} - 1.74 \zeta_{m}$

- Considering $\Omega_m$ as a parameter:

Present-day drift rate for the Bekenstein model:

${\left( \frac{1}{H} \frac{\dot{\alpha}}{\alpha} \right)}_0 = - \zeta_{\Lambda} + 2(\zeta_{\Lambda} -2 \zeta_{m}) \frac{\Omega_m}{\sqrt{\Omega_{\Lambda}}} \log{\frac{1 + \sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_{m}}}}\Leftrightarrow$

$\Leftrightarrow {\left( \frac{1}{H} \frac{\dot{\alpha}}{\alpha} \right)}_0 = \left( 2 \frac{\Omega_m}{\sqrt{\Omega_{\Lambda}}} \log{\frac{1 + \sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_{m}}}} - 1\right)\zeta_{\Lambda} - \left(4 \frac{\Omega_m}{\sqrt{\Omega_{\Lambda}}} \log{\frac{1 + \sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_{m}}}}\right)  \zeta_{m}$


Present-day drift rate for the CPL Parametrization:

$ {\left( \frac{1}{H} \frac{\dot{\alpha}}{\alpha} \right)}_0 = - \zeta \sqrt{3 {\Omega_{\phi}}_0 |1+w_0|}$

where ${\Omega_{\phi}}_0  = \frac{\Omega_{\Lambda}}{\Omega_{\Lambda} + \Omega_m \times 1^{-3(w_0 + w_a)}}$

For a flat universe we have: $\Omega_m + \Omega_{\Lambda} = 1 \Rightarrow \Omega_{\Lambda} = 1 - \Omega_m$

- Constraint on current drift of $\alpha$ by R. Lange:

$\left( \frac{1}{H}\frac{\dot{\alpha}}{\alpha} \right)_{z=0} = 0.014 \pm 0.015 ppm$

In [6]:
def AtomicClocks(OmegaM, Zm, Zl, w0, zeta, model):
    
    #Constraint R.Lange in ppm
    unc_RLange = .015
    
    if model == 'Bekenstein': #OmegaM, Zm, Zl

        AtomicClocks = np.zeros((3,3))

        PriorOmegaM1 = (Zl - (2*Zm)) * ((OmegaM*np.log(OmegaM)) - (2*OmegaM*np.log(np.sqrt(-OmegaM+1)+1)) + (4*np.log(np.sqrt(-OmegaM+1)+1)) - (2*np.log(OmegaM)) - (2*np.sqrt(-OmegaM+1)))
        PriorOmegaM2 = 2 * (-OmegaM+1) * (np.sqrt(-OmegaM+1))
        A = (PriorOmegaM1/PriorOmegaM2)
        
        B = -1 * (((4*OmegaM)*((np.log(np.sqrt(-OmegaM+1)+1)-((1/2)*(np.log(OmegaM)))))/(np.sqrt(-OmegaM+1))))
        
        C = -1 + (((2*OmegaM * np.log(1 + np.sqrt(1-OmegaM)))-(OmegaM*np.log(OmegaM)))/(np.sqrt(1-OmegaM)))

        AtomicClocks  = np.array([[A**2*unc_RLange**(-2), A*B*unc_RLange**(-2), A*C*unc_RLange**(-2)],
                                  [B*A*unc_RLange**(-2), B**2*unc_RLange**(-2), B*C*unc_RLange**(-2)],
                                  [C*A*unc_RLange**(-2), C*B*unc_RLange**(-2), C**2*unc_RLange**(-2)]], dtype='float64')

    
    if model == 'CPL': #OmegaM, w0, zeta
        
        AtomicClocks = np.zeros((3,3))
        
        PriorOmegaM1 = -1 * np.sqrt(3) * zeta * (-1-w0)
        PriorOmegaM2 = 2 * np.sqrt((1 - OmegaM)*(1+w0))
        
        A = PriorOmegaM1/PriorOmegaM2
        
        Priorw01 = -1 * np.sqrt(3) * zeta * np.sqrt(1 - OmegaM)
        Priorw02 = 2 * np.sqrt(1+w0)
        
        B = Priorw01/Priorw02
        
        C = -1 * np.sqrt(3) * np.sqrt((1 - OmegaM)*(1+w0))
        
        AtomicClocks  = np.array([[A**2*unc_RLange**(-2), A*B*unc_RLange**(-2), A*C*unc_RLange**(-2)],
                                  [B*A*unc_RLange**(-2), B**2*unc_RLange**(-2), B*C*unc_RLange**(-2)],
                                  [C*A*unc_RLange**(-2), C*B*unc_RLange**(-2), C**2*unc_RLange**(-2)]], dtype='float64')

    
    return AtomicClocks

#### Separate Priors

In [7]:
unch = 0.05
uncOmegaM = 0.03
uncw0 = 0.1
uncwa = 0.4

In [8]:
PriorClocksBEK0 = AtomicClocks(OmegaM=0.3, Zm=0., Zl=0., w0=0., zeta=0., model='Bekenstein')
PriorBEK0 = add1DPriors(PriorClocksBEK0, [[uncOmegaM,0]])

PriorClocksBEK = AtomicClocks(OmegaM=0.3, Zm=0., Zl=0.1, w0=0., zeta=0., model='Bekenstein')
PriorBEK = add1DPriors(PriorClocksBEK, [[uncOmegaM,0]])

PriorClocksCPLT = AtomicClocks(OmegaM=0.3, Zm=0., Zl=0., w0=-0.8, zeta=-0.15, model='CPL')
PriorCPLT = np.copy(PriorClocksCPLT)
PriorCPLT = np.insert(PriorCPLT, 2, 0, axis=0)
PriorCPLT = np.insert(PriorCPLT, 2, 0, axis=1)
PriorCPLT = add1DPriors(PriorCPLT, [[uncOmegaM,0],[uncw0,1],[uncwa,2]])


PriorClocksCPLF = AtomicClocks(OmegaM=0.3, Zm=0., Zl=0., w0=-0.9, zeta=-0.15, model='CPL')
PriorCPLF = np.copy(PriorClocksCPLF)
PriorCPLF = np.insert(PriorCPLF, 2, 0, axis=0)
PriorCPLF = np.insert(PriorCPLF, 2, 0, axis=1)
PriorCPLF = add1DPriors(PriorCPLF, [[uncOmegaM,0],[uncw0,1],[uncwa,2]])

# Measurements
### - Redshifts

In [9]:
#ANDES Baseline
zAndes_15baseline = np.linspace(1,2.5,num=15)
zAndes_10baseline = np.linspace(2.5,4,num=10)


#ANDES Optimistic
zAndes_20optimistic = np.linspace(1,2.5,num=20)
zAndes_15optimistic1 = np.linspace(2.5,4,num=15)
zAndes_15optimistic2 = np.linspace(4,7,num=15)

### - Uncertainties

In [10]:
#ANDES Baseline (in ppm)
uncAndes_15baseline = np.zeros(15) + (0.1)
uncAndes_10baseline = np.zeros(10) + (0.2)


#ANDES Optimistic (in ppm)
uncAndes_20optimistic = np.zeros(20) + (0.05)
uncAndes_15optimistic1 = np.zeros(15) + (0.1)
uncAndes_15optimistic2 = np.zeros(15) + (1)

# Fisher Matrices
## - No Priors
### Bekenstein
#### BEK0 $(\zeta_\Lambda = \zeta_m = 0)$

In [11]:
FM_BEK0_Andes_15baseline    = FisherMatrix_ab(zAndes_15baseline, uncAndes_15baseline, OmegaM, w0=None, wa=None, Zl=0, Zm=0, OCP= 'parameter', analysis = 'alpha')
FM_BEK0_Andes_10baseline    = FisherMatrix_ab(zAndes_10baseline, uncAndes_10baseline, OmegaM, w0=None, wa=None, Zl=0, Zm=0, OCP= 'parameter', analysis = 'alpha')

FM_BEK0_Andes_20optimistic  = FisherMatrix_ab(zAndes_20optimistic, uncAndes_20optimistic, OmegaM, w0=None, wa=None, Zl=0, Zm=0, OCP= 'parameter', analysis = 'alpha')
FM_BEK0_Andes_15optimistic1 = FisherMatrix_ab(zAndes_15optimistic1, uncAndes_15optimistic1, OmegaM, w0=None, wa=None, Zl=0, Zm=0, OCP= 'parameter', analysis = 'alpha')
FM_BEK0_Andes_15optimistic2 = FisherMatrix_ab(zAndes_15optimistic2, uncAndes_15optimistic2, OmegaM, w0=None, wa=None, Zl=0, Zm=0, OCP= 'parameter', analysis = 'alpha')


BEK0_AlphaB_np = FM_BEK0_Andes_baseline = FM_BEK0_Andes_15baseline + FM_BEK0_Andes_10baseline
BEK0_AlphaO_np = FM_BEK0_Andes_optimistic = FM_BEK0_Andes_20optimistic + FM_BEK0_Andes_15optimistic1 + FM_BEK0_Andes_15optimistic2

#### BEK($\zeta_m = 0$ and $\zeta_\Lambda = +0.1$ppm)

In [12]:
FM_BEK_Andes_15baseline    = FisherMatrix_ab(zAndes_15baseline, uncAndes_15baseline, OmegaM, w0=None, wa=None, Zl=+0.1, Zm=0, OCP= 'parameter', analysis = 'alpha')
FM_BEK_Andes_10baseline    = FisherMatrix_ab(zAndes_10baseline, uncAndes_10baseline, OmegaM, w0=None, wa=None, Zl=+0.1, Zm=0, OCP= 'parameter', analysis = 'alpha')

FM_BEK_Andes_20optimistic  = FisherMatrix_ab(zAndes_20optimistic, uncAndes_20optimistic, OmegaM, w0=None, wa=None, Zl=+0.1, Zm=0, OCP= 'parameter', analysis = 'alpha')
FM_BEK_Andes_15optimistic1 = FisherMatrix_ab(zAndes_15optimistic1, uncAndes_15optimistic1, OmegaM, w0=None, wa=None, Zl=+0.1, Zm=0, OCP= 'parameter', analysis = 'alpha')
FM_BEK_Andes_15optimistic2 = FisherMatrix_ab(zAndes_15optimistic2, uncAndes_15optimistic2, OmegaM, w0=None, wa=None, Zl=+0.1, Zm=0, OCP= 'parameter', analysis = 'alpha')


BEK_AlphaB_np = FM_BEK_Andes_15baseline + FM_BEK_Andes_10baseline
BEK_AlphaO_np = FM_BEK_Andes_20optimistic + FM_BEK_Andes_15optimistic1 + FM_BEK_Andes_15optimistic2

### CPL Parametrization
#### CPLT ($w_0 = -0.8$, $w_a = -0.2$ and $\zeta = -0.15$ppm)

In [13]:
FM_CPL_Andes_15baselineT = FisherMatrix_aCPL(zAndes_15baseline, uncAndes_15baseline, OmegaM=0.3, w0=-0.8, wa=-0.2, zeta=-0.15, analysis = 'alpha')
FM_CPL_Andes_10baselineT = FisherMatrix_aCPL(zAndes_10baseline, uncAndes_10baseline, OmegaM=0.3, w0=-0.8, wa=-0.2, zeta=-0.15, analysis = 'alpha')

FM_CPL_Andes_20optimisticT = FisherMatrix_aCPL(zAndes_20optimistic, uncAndes_20optimistic, OmegaM=0.3, w0=-0.8, wa=-0.2, zeta=-0.15, analysis = 'alpha')
FM_CPL_Andes_15optimistic1T = FisherMatrix_aCPL(zAndes_15optimistic1, uncAndes_15optimistic1, OmegaM=0.3, w0=-0.8, wa=-0.2, zeta=-0.15, analysis = 'alpha')
FM_CPL_Andes_15optimistic2T = FisherMatrix_aCPL(zAndes_15optimistic2, uncAndes_15optimistic2, OmegaM=0.3, w0=-0.8, wa=-0.2, zeta=-0.15, analysis = 'alpha')


FM_CPL_AlphaB_T_np = FM_CPL_Andes_15baselineT + FM_CPL_Andes_10baselineT
FM_CPL_AlphaO_T_np = FM_CPL_Andes_20optimisticT + FM_CPL_Andes_15optimistic1T + FM_CPL_Andes_15optimistic2T

#### CPLF ($w_0 = -0.9$, $w_a = +0.3$ and $\zeta = -0.15$ppm)

In [14]:
FM_CPL_Andes_15baselineF = FisherMatrix_aCPL(zAndes_15baseline, uncAndes_15baseline, OmegaM=0.3, w0=-0.9, wa=0.3, zeta=-0.15, analysis = 'alpha')
FM_CPL_Andes_10baselineF = FisherMatrix_aCPL(zAndes_10baseline, uncAndes_10baseline, OmegaM=0.3, w0=-0.9, wa=0.3, zeta=-0.15, analysis = 'alpha')

FM_CPL_Andes_20optimisticF = FisherMatrix_aCPL(zAndes_20optimistic, uncAndes_20optimistic, OmegaM=0.3, w0=-0.9, wa=0.3, zeta=-0.15, analysis = 'alpha')
FM_CPL_Andes_15optimistic1F = FisherMatrix_aCPL(zAndes_15optimistic1, uncAndes_15optimistic1, OmegaM=0.3, w0=-0.9, wa=0.3, zeta=-0.15, analysis = 'alpha')
FM_CPL_Andes_15optimistic2F = FisherMatrix_aCPL(zAndes_15optimistic2, uncAndes_15optimistic2, OmegaM=0.3, w0=-0.9, wa=0.3, zeta=-0.15, analysis = 'alpha')


FM_CPL_AlphaB_F_np = FM_CPL_Andes_15baselineF + FM_CPL_Andes_10baselineF
FM_CPL_AlphaO_F_np = FM_CPL_Andes_20optimisticF + FM_CPL_Andes_15optimistic1F + FM_CPL_Andes_15optimistic2F

## - With Priors
### Bekenstein
#### BEK0 ($\zeta_\Lambda = \zeta_m = 0$)

In [15]:
BEK0_AlphaB = BEK0_AlphaB_np + PriorBEK0
BEK0_AlphaO = BEK0_AlphaO_np + PriorBEK0

#### BEK ($\zeta_m = 0$ and $\zeta_\Lambda = +0.1$ppm)

In [16]:
BEK_AlphaB = BEK_AlphaB_np + PriorBEK
BEK_AlphaO = BEK_AlphaO_np + PriorBEK

#### CPLT ($w_0 = -0.8$, $w_a = -0.2$ and $\zeta = -0.15$ppm)

In [17]:
CPL_AlphaB_T = FM_CPL_AlphaB_T_np + PriorCPLT
CPL_AlphaO_T = FM_CPL_AlphaO_T_np + PriorCPLT

#### CPLF ($w_0 = -0.9$, $w_a = +0.3$ and $\zeta = -0.15$ppm)

In [18]:
CPL_AlphaB_F = FM_CPL_AlphaB_F_np + PriorCPLF
CPL_AlphaO_F = FM_CPL_AlphaO_F_np + PriorCPLF

# Confidence ellipses (1-sigma)

#### Bekenstein

In [19]:
def EllipsesAlphaBek(parameter1,parameter2, parameter_values, Fs, ells_color, ells_linestyle, Fs_legend=None): 
    ells = []
    custom_lines=[]
    if not Fs_legend:
        Fs_legend = len(Fs)*[None]
    for i in range(len(Fs)):
        parameter1_value, parameter2_value = parameter_values[i] 
        
        if parameter1 == 'OmegaM':
            parameter1_legend = '$\Omega_m$'
        
            if parameter2 == 'Zm':
                Fm = marginalize(Fs[i],'Zl',analysis='alpha-BEK')
                parameter2_legend = '$\zeta_m$ (ppm)'
                
            if parameter2 == 'Zl':
                Fm = marginalize(Fs[i],'Zm',analysis='alpha-BEK')
                parameter2_legend = '$\zeta_\Lambda$ (ppm)'  
           
        
        if parameter1 == 'Zm' and parameter2 == 'Zl':
            parameter1_legend = '$\zeta_m$ (ppm)'
            parameter2_legend = '$\zeta_\Lambda$ (ppm)'
            Fm = marginalize(Fs[i],'OmegaM',analysis='alpha-BEK')
        
        
        Pa = preelipse(Fm)
        color, name, style,facecolor, alpha = ells_color[i], Fs_legend[i], ells_linestyle[i], 'none', 1
        ells.append([parameter1_value,parameter2_value,Pa[0],Pa[1],Pa[2],name,color,style,facecolor, alpha])
        custom_lines+= [Line2D([0], [0], color=ells_color[i], lw=1.1, ls=ells_linestyle[i])]
        
                            
    ellipses(ells,[parameter1_legend,parameter2_legend])
    plt.tick_params(axis='both', which='major', labelsize=16)
    if Fs_legend[0]: 
        plt.legend(custom_lines, Fs_legend)
    
    #Definition of plot limits and figure name
    #plt.xlim(-1.1,-0.7)
    #plt.ylim(-0.4, 1.0)
    #plt.savefig('name.png', dpi=300, bbox_inches='tight')

    plt.show()

The possible combinations are: (Zm,Zl), (OmegaM, Zm), (OmegaM, Zl)

Specify the <b>combination of parameters</b>, <b>fiducial values</b> of the model, the <b>matrices</b>, <b>colours</b>, <b>linestyle</b> and <b>names</b> for the legend.

In [20]:
#EllipsesAlphaBek('Zm','Zl',[[0.0,0.0],[0.0,0.1]],[BEK0_AlphaB, BEK_AlphaB], ['blue','red'], ['-', (0, (8,8))],['$\zeta_\Lambda$=0.0 ppm', '$\zeta_\Lambda$=+0.1 ppm'])

#### CPL

In [21]:
def EllipsesAlphaCPL(parameter1,parameter2, parameter_values, Fs, ells_color, ells_linestyle, Fs_legend=None): 
    ells = []
    custom_lines=[]
    if not Fs_legend:
        Fs_legend = len(Fs)*[None]
    for i in range(len(Fs)):
        parameter1_value, parameter2_value = parameter_values[i] 
        
        if parameter1 == 'OmegaM':
            parameter1_legend = '$\Omega_m$'
        
            if parameter2 == 'zeta':
                Fm = marginalize(Fs[i],'wa',analysis='alpha-CPL')
                Fm = marginalize(Fm,'w0',analysis='alpha-CPL')
                parameter2_legend = '$\zeta$ (ppm)'
                
            if parameter2 == 'w0':
                Fm = marginalize(Fs[i],'zeta',analysis='alpha-CPL')
                Fm = marginalize(Fm,'wa',analysis='alpha-CPL')
                parameter2_legend = '$w_0$'
                
            if parameter2 == 'wa':
                Fm = marginalize(Fs[i],'zeta',analysis='alpha-CPL')
                Fm = marginalize(Fm,'w0',analysis='alpha-CPL')
                parameter2_legend = '$w_a$'
                
                
        if parameter1 == 'w0':
            parameter1_legend = '$w_0$'
            
            if parameter2 == 'zeta':
                Fm = marginalize(Fs[i],'wa',analysis='alpha-CPL')
                Fm = marginalize(Fm,'OmegaM',analysis='alpha-CPL')
                parameter2_legend = '$\zeta$ (ppm)'
                
            if parameter2 == 'wa':
                Fm = marginalize(Fs[i],'zeta',analysis='alpha-CPL')
                Fm = marginalize(Fm,'OmegaM',analysis='alpha-CPL')
                parameter2_legend = '$w_a$'
                
        if parameter1 == 'wa' and parameter2 == 'zeta':
            parameter1_legend = '$w_a$'
            parameter2_legend = '$\zeta$ (ppm)'
            Fm = marginalize(Fs[i],'w0',analysis='alpha-CPL')
            Fm = marginalize(Fm,'OmegaM',analysis='alpha-CPL')
                
           
        Pa = preelipse(Fm)
        color, name, style,facecolor, alpha = ells_color[i], Fs_legend[i], ells_linestyle[i], 'none', 1
        ells.append([parameter1_value,parameter2_value,Pa[0],Pa[1],Pa[2],name,color,style,facecolor, alpha])
        custom_lines+= [Line2D([0], [0], color=ells_color[i], lw=1.1, ls=ells_linestyle[i])]
        
                            
    ellipses(ells,[parameter1_legend,parameter2_legend])
    plt.tick_params(axis='both', which='major', labelsize=16)
    if Fs_legend[0]: 
        plt.legend(custom_lines, Fs_legend)
    
    #Definition of plot limits and figure name
    #plt.xlim(-1.1,-0.7)
    #plt.ylim(-0.4, 1.0)
    #plt.savefig('name.png', dpi=300, bbox_inches='tight')

    plt.show()

The possible combinations are: (OmegaM, zeta), (OmegaM, w0), (OmegaM, wa), (w0, zeta), (w0, wa), (wa, zeta)

Specify the <b>combination of parameters</b>, <b>fiducial values</b> of the model, the <b>matrices</b>, <b>colours</b>, <b>linestyle</b> and <b>names</b> for the legend.

In [22]:
#EllipsesAlphaCPL('OmegaM','zeta',[[0.3,-0.15],[0.3,-0.15]],[CPL_AlphaB_T, CPL_AlphaB_F], ['blue','red'], ['-', (0, (8,8))],['Thawing', 'Freezing'])

# Table with ρ, FoM and σ

#### Bekenstein

In [23]:
def TableAlphaBek(matrices):
    i=0
    df=pd.DataFrame()
    nformat = "{:.0f}"
    float_format="{:.3f}"

    for matrix in matrices:
        rows = {'ρ(Ωm, Zm)':float_format.format(FigMeriteRho(matrix,['Zl'],analysis='alpha-BEK')[1]),
                'ρ(Ωm, Zl)':float_format.format(FigMeriteRho(matrix,['Zm'],analysis='alpha-BEK')[1]),
                'ρ(Zm, Zl)':float_format.format(FigMeriteRho(matrix,['OmegaM'],analysis='alpha-BEK')[1]),                      
                'FoM(Ωm, Zm)': nformat.format(FigMeriteRho(matrix,['Zl'],analysis='alpha-BEK')[0]),
                'FoM(Ωm, Zl)': nformat.format(FigMeriteRho(matrix,['Zm'],analysis='alpha-BEK')[0]),
                'FoM(Zm, Zl)': nformat.format(FigMeriteRho(matrix,['OmegaM'],analysis='alpha-BEK')[0]),
                'σ(Ωm)':float_format.format(Sigmas(matrix)[0]),
                'σ(Zm)':float_format.format(Sigmas(matrix)[1]),
                'σ(Zl)':float_format.format(Sigmas(matrix)[2])}
        i+=1
        df = df.append(rows,ignore_index=True)
        del rows
    
    
    Datasets = ['Baseline BEK0','Baseline BEK','Optimistic BEK0', 'Optimistic BEK']
    df.insert(0, "Parameters", Datasets, True)
    
    df = df.transpose()
    
            
    return df

In [28]:
matricesBEK = [BEK0_AlphaB, BEK_AlphaB, BEK0_AlphaO, BEK_AlphaO]
#tab = TableAlphaBek(matricesBEK)

#See the table in DataFrame:
#tab

#Transform dataframe to latex code:
#print(tab.to_latex(header=False,index=True))

#### CPL

In [25]:
def TableAlphaCPL(matrices):
    i=0
    df=pd.DataFrame()
    nformat = "{:.0f}"
    float_format="{:.3f}"
    
    for matrix in matrices:
        rows = {'ρ(Ωm, w0)':float_format.format(FigMeriteRho(matrix,['wa','zeta'],analysis='alpha-CPL')[1]),
                'ρ(Ωm, wa)':float_format.format(FigMeriteRho(matrix,['w0','zeta'],analysis='alpha-CPL')[1]),
                'ρ(Ωm, zeta)':float_format.format(FigMeriteRho(matrix,['w0','wa'],analysis='alpha-CPL')[1]),                       
                'ρ(w0, wa)':float_format.format(FigMeriteRho(matrix,['OmegaM','zeta'],analysis='alpha-CPL')[1]),
                'ρ(w0, zeta)':float_format.format(FigMeriteRho(matrix,['OmegaM','wa'],analysis='alpha-CPL')[1]),
                'ρ(wa, zeta)':float_format.format(FigMeriteRho(matrix,['OmegaM','w0'],analysis='alpha-CPL')[1]),
                'FoM(Ωm, w0)': nformat.format(FigMeriteRho(matrix,['wa','zeta'],analysis='alpha-CPL')[0]),
                'FoM(Ωm, wa)': nformat.format(FigMeriteRho(matrix,['w0','zeta'],analysis='alpha-CPL')[0]),
                'FoM(Ωm, zeta)': nformat.format(FigMeriteRho(matrix,['w0','wa'],analysis='alpha-CPL')[0]),
                'FoM(w0, wa)': nformat.format(FigMeriteRho(matrix,['OmegaM','zeta'],analysis='alpha-CPL')[0]),
                'FoM(w0, zeta)': nformat.format(FigMeriteRho(matrix,['OmegaM','wa'],analysis='alpha-CPL')[0]),
                'FoM(wa, zeta)': nformat.format(FigMeriteRho(matrix,['OmegaM','w0'],analysis='alpha-CPL')[0]),
                'σ(Ωm)':float_format.format(Sigmas(matrix)[0]),
                'σ(w0)':float_format.format(Sigmas(matrix)[1]),
                'σ(wa)':float_format.format(Sigmas(matrix)[2]),
                'σ(zeta)':float_format.format(Sigmas(matrix)[3])}
        i+=1
        df = df.append(rows,ignore_index=True)
        del rows
    
    
    Datasets = ['Baseline Thawing','Baseline Freezing', 'Optimistic Thawing','Optimistic Freezing']
    df.insert(0, "Parameters", Datasets, True)
    
    df = df.transpose()
    
            
    return df

In [27]:
matricesCPL = [CPL_AlphaB_T, CPL_AlphaB_F, CPL_AlphaO_T, CPL_AlphaO_F]
#tab = TableAlphaCPL(matricesCPL)

#See the table in DataFrame:
#tab

#Transform dataframe to latex code:
#print(tab.to_latex(header=False,index=True))